# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 12
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr12.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr12.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 389 389

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000120645,ENSG00000111181,ENSG00000171840,ENSG00000060237,ENSG00000002016,ENSG00000082805,ENSG00000111186,ENSG00000006831,ENSG00000151062,ENSG00000151065
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000120645,1.000000000,-0.049416899,-0.056575107,0.048658608,0.02922662,0.000989973,0.03911587,-0.04769783,-0.018081879,-0.068448015
ENSG00000111181,-0.049416899,1.000000000,-0.021797121,0.005337499,0.06554360,-0.024246285,0.10982490,-0.05659368,0.004522582,-0.012559306
ENSG00000171840,-0.056575107,-0.021797121,1.000000000,-0.009131751,-0.13907795,-0.074546131,0.04276620,0.11273593,0.050262549,-0.007609613
ENSG00000060237,0.048658608,0.005337499,-0.009131751,1.000000000,-0.03916614,0.023379811,-0.11364707,-0.02096805,-0.026575339,0.029822830
ENSG00000002016,0.029226625,0.065543598,-0.139077949,-0.039166141,1.00000000,0.021312729,-0.12037136,-0.05725021,0.112585641,0.056742248
ENSG00000082805,0.000989973,-0.024246285,-0.074546131,0.023379811,0.02131273,1.000000000,0.01840896,-0.06007252,0.079096633,0.052220807


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 44

[1] -0.007044259 -0.010513918 -0.032729471 -0.061794686 -0.064990525
 [6] -0.069403653 -0.090145985 -0.114621464 -0.119831352 -0.126113556
[11] -0.198365847 -0.204057608 -0.218171601 -0.234519924 -0.252332821
[16] -0.275437678 -0.298224726 -0.308873224 -0.324756500 -0.367133110
[21] -0.397989538 -0.417515248 -0.468577841 -0.483196498 -0.579933045
[26] -0.584894838 -0.589107894 -0.608419527 -0.625252885 -0.763448648
[31] -0.882663636 -0.943158023 -1.202862277 -1.241225879 -1.249934410
[36] -1.336640807 -1.398206603 -1.411206263 -1.806602801 -1.853229386
[41] -1.918589528 -2.281141528 -2.611284934 -3.597278807

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 389 389

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000120645,ENSG00000111181,ENSG00000171840,ENSG00000060237,ENSG00000002016,ENSG00000082805,ENSG00000111186,ENSG00000006831,ENSG00000151062,ENSG00000151065
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000120645,1.000000000,-0.049416899,-0.056575107,0.048658608,0.02922662,0.000989973,0.03911587,-0.04769783,-0.018081879,-0.068448015
ENSG00000111181,-0.049416899,1.000000000,-0.021797121,0.005337499,0.06554360,-0.024246285,0.10982490,-0.05659368,0.004522582,-0.012559306
ENSG00000171840,-0.056575107,-0.021797121,1.000000000,-0.009131751,-0.13907795,-0.074546131,0.04276620,0.11273593,0.050262549,-0.007609613
ENSG00000060237,0.048658608,0.005337499,-0.009131751,1.000000000,-0.03916614,0.023379811,-0.11364707,-0.02096805,-0.026575339,0.029822830
ENSG00000002016,0.029226625,0.065543598,-0.139077949,-0.039166141,1.00000000,0.021312729,-0.12037136,-0.05725021,0.112585641,0.056742248
ENSG00000082805,0.000989973,-0.024246285,-0.074546131,0.023379811,0.02131273,1.000000000,0.01840896,-0.06007252,0.079096633,0.052220807


,ENSG00000120645,ENSG00000111181,ENSG00000171840,ENSG00000060237,ENSG00000002016,ENSG00000082805,ENSG00000111186,ENSG00000006831,ENSG00000151062,ENSG00000151065
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000120645,1.00000000,-0.005444680,-0.02489953,0.010450514,0.018177695,0.011039772,0.01242243,-0.04305986,-0.022686539,-0.031379025
ENSG00000111181,-0.00544468,1.000000000,-0.01209668,-0.040424638,0.030789176,-0.001935039,0.04153581,-0.01872605,-0.003911105,0.014576503
ENSG00000171840,-0.02489953,-0.012096682,1.00000000,-0.017831006,-0.079905304,-0.048398897,0.03334152,0.07425710,0.020306092,-0.024292982
ENSG00000060237,0.01045051,-0.040424638,-0.01783101,1.000000000,0.004035556,0.001755507,-0.04831061,-0.02612281,-0.027971074,0.008633825
ENSG00000002016,0.01817770,0.030789176,-0.07990530,0.004035556,1.000000000,0.026340178,-0.04927355,-0.04255480,0.038055594,0.037793600
ENSG00000082805,0.01103977,-0.001935039,-0.04839890,0.001755507,0.026340178,1.000000000,0.02119234,-0.02699893,0.005606310,0.032120609


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)